In [ ]:
import os
os.listdir("../input/stage2wdatafiles")

In [ ]:
import numpy as np
import pandas as pd
from sklearn import *

teams = pd.read_csv('../input/stage2wdatafiles/WTeams.csv')
teams2 = pd.read_csv('../input/stage2wdatafiles/WTeamSpellings.csv', encoding='latin-1')

season_dresults = pd.read_csv('../input/stage2wdatafiles/WRegularSeasonDetailedResults.csv')
season_cresults = pd.read_csv('../input/stage2wdatafiles/WRegularSeasonCompactResults.csv')
#tourney_dresults = pd.read_csv('../input/wdatafiles/WNCAATourneyDetailedResults.csv')



In [ ]:
season_cresults.describe()

In [ ]:
teams2 = teams2.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
teams2.columns = ['TeamID', 'TeamNameCount']
teams = pd.merge(teams, teams2, how='left', on=['TeamID'])
del teams2

In [ ]:
import matplotlib.pyplot as plt

for y in set(season_dresults["Season"].ravel()):
    plt.hist(season_dresults.loc[(season_dresults["Season"]==y),"DayNum"], bins=132)

plt.show()

In [ ]:
season_dresults.columns

season_dresults['ST'] = 'S'

stats = ["TeamID", "Season", "Games", "Wins", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF", "FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]

# Collect stats for each team for each season
frame = np.zeros([len(teams)*9, len(stats)])
season_stats = pd.DataFrame(frame, index=range(len(teams)*9), columns=stats)
season_stats["TeamID"] = [item for sublist in [9*[t_id] for t_id in teams["TeamID"]] for item in sublist]
season_stats["Season"] = len(teams)*[x for x in range(2010,2019)]

for c,r in season_dresults.iterrows():
    
    wstats = r[["W"+x for x in ["TeamID", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]]]
    lstats = r[["L"+x for x in ["TeamID", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]]]
    
    #print(wstats[["WTeamID", "WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values)
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] += wstats[["WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]] += lstats[["LFGM", "LFGA", "LFGM3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]].values   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] += lstats[["LFGM", "LFGA", "LFGM3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]].values
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]] += wstats[["WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values    
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]), ["Games", "Wins"]] += 1   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]), "Games"] += 1   
    
  #  print(season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"])])
  #  break
    
season_stats = season_stats.astype("float64")
season_stats.to_csv("SeasonStats.csv")

#season_stats.groupby(["TeamID", "Season"]).max()

In [ ]:
season_dresults = season_dresults.loc[season_dresults["DayNum"]>100]
season_cresults = season_cresults.loc[season_cresults["DayNum"]>100]

stats = ["TeamID", "Season", "Games", "Wins", "Score","FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF", "Score_A", "FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]

# Collect stats for each team for each season
frame = np.zeros([len(teams)*len(range(1998,2020)), len(stats)])
season_stats = pd.DataFrame(frame, index=range(len(teams)*len(range(1998,2020))), columns=stats)
season_stats["TeamID"] = [item for sublist in [len(range(1998,2020))*[t_id] for t_id in teams["TeamID"]] for item in sublist]
season_stats["Season"] = len(teams)*[x for x in range(1998,2020)]

for c,r in season_dresults.iterrows():
    
    wstats = r[["W"+x for x in ["TeamID", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]]]
    lstats = r[["L"+x for x in ["TeamID", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]]]
    
    #print(wstats[["WTeamID", "WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values)
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] += wstats[["WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]] += lstats[["LFGM", "LFGA", "LFGM3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]].values   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] += lstats[["LFGM", "LFGA", "LFGM3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]].values
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]] += wstats[["WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values    
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]), ["Games", "Wins"]] += 1   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]), "Games"] += 1   

    
for c,r in season_cresults.iterrows():
    
    wstats = r[["W"+x for x in ["TeamID", "Score"]]]
    lstats = r[["L"+x for x in ["TeamID", "Score"]]]
    
    #print(wstats[["WTeamID", "WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values)
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     "Score"] += wstats["WScore"]
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     "Score_A"] += lstats["LScore"]   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     "Score"] += lstats["LScore"]
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     "Score_A"] += wstats["WScore"]   
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]), ["Games", "Wins"]] += 1   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]), "Games"] += 1   
        
  #  print(season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"])])
  #  break

season_stats.loc[season_stats["Season"].isin(range(1998,2010))] = season_stats.loc[season_stats["Season"].isin(range(1998,2010))].replace({0:np.nan})
season_stats = season_stats.astype("float64")
season_stats.to_csv("RecentStatsSince1998.csv")


season_dresults = pd.read_csv('../input/wdatafiles/WRegularSeasonDetailedResults.csv')

fake_tourney = season_dresults.loc[season_dresults["DayNum"]>100]
fake_tourney.to_csv("faketourney_dresults.csv", index=False)

season_dresults = season_dresults.loc[season_dresults["DayNum"]>100]


season_dresults['ST'] = 'S'

stats = ["TeamID", "Season", "Games", "Wins", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF", "FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]

# Collect stats for each team for each season
frame = np.zeros([len(teams)*9, len(stats)])
season_stats = pd.DataFrame(frame, index=range(len(teams)*9), columns=stats)
season_stats["TeamID"] = [item for sublist in [9*[t_id] for t_id in teams["TeamID"]] for item in sublist]
season_stats["Season"] = len(teams)*[x for x in range(2010,2019)]

for c,r in season_dresults.iterrows():
    
    wstats = r[["W"+x for x in ["TeamID", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]]]
    lstats = r[["L"+x for x in ["TeamID", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]]]
    
    #print(wstats[["WTeamID", "WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values)
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] += wstats[["WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]] += lstats[["LFGM", "LFGA", "LFGM3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]].values   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] += lstats[["LFGM", "LFGA", "LFGM3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]].values
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]] += wstats[["WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values    
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]), ["Games", "Wins"]] += 1   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]), "Games"] += 1   
    
  #  print(season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"])])
  #  break
    
season_stats = season_stats.astype("float64")
season_stats.to_csv("FakeSeasonStats.csv")

#season_stats.groupby(["TeamID", "Season"]).max()

season_dresults = season_dresults.loc[season_dresults["DayNum"]>70]

stats = ["TeamID", "Season", "Games", "Wins", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF", "FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]

# Collect stats for each team for each season
frame = np.zeros([len(teams)*9, len(stats)])
season_stats = pd.DataFrame(frame, index=range(len(teams)*9), columns=stats)
season_stats["TeamID"] = [item for sublist in [9*[t_id] for t_id in teams["TeamID"]] for item in sublist]
season_stats["Season"] = len(teams)*[x for x in range(2010,2019)]

for c,r in season_dresults.iterrows():
    
    wstats = r[["W"+x for x in ["TeamID", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]]]
    lstats = r[["L"+x for x in ["TeamID", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]]]
    
    #print(wstats[["WTeamID", "WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values)
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] += wstats[["WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]] += lstats[["LFGM", "LFGA", "LFGM3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]].values   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] += lstats[["LFGM", "LFGA", "LFGM3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]].values
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM_A", "FGA_A", "FGM3_A", "FTM_A", "FTA_A", "OR_A", "DR_A", "Ast_A", "TO_A", "Stl_A", "Blk_A", "PF_A"]] += wstats[["WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values    
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]), ["Games", "Wins"]] += 1   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]), "Games"] += 1   
    
  #  print(season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"])])
  #  break
    
season_stats = season_stats.astype("float64")
season_stats.to_csv("FakeRecentStats.csv")


from copy import deepcopy
ss = deepcopy(season_stats)

ss[["Wins", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] = ss[["Wins", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]].values/np.reshape(ss["Games"].values, [-1,1])



s = pd.DataFrame([[0,1,3,5],[0,2,7,3],[1,2,8,2],[0,1,1,0]], columns=["WTeamID", "LTeamID", "WG", "LG"])

def winner(r, dt):
    

s2 = pd.DataFrame(np.zeros([3,4]), columns=["ID", "Games", "Wins", "Goals"])
s.apply(lambda x: winner(x["WTeamID"], x[""]))

stats = ["TeamID", "Season", "Games", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]

from copy import deepcopy


season_dresults["GameID"] = np.arange(len(season_dresults), dtype="int16")

season_games = pd.DataFrame(numpy.zeros([len(season_dresults)])

                            
                            
frame = np.zeros([len(teams)*9, len(stats)])
frame[:] = np.nan
                            
# Collect stats for each team for each season
season_stats = pd.DataFrame(np.zeros([len(teams)*9, len(stats)]), index=range(len(teams)*9), columns=stats)
season_stats["TeamID"] = [item for sublist in [9*[t_id] for t_id in teams["TeamID"]] for item in sublist]
season_stats["Season"] = len(teams)*[x for x in range(2010,2019)]

for c,r in season_dresults.iterrows():
    
    wstats = r[["W"+x for x in ["TeamID", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]]]
    lstats = r[["L"+x for x in ["TeamID", "FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]]]
    
    #print(wstats[["WTeamID", "WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values)
    
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] += wstats[["WFGM", "WFGA", "WFGM3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]),
                     ["FGM", "FGA", "FGM3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]] += lstats[["LFGM", "LFGA", "LFGM3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]].values
    season_stats.loc[(season_stats.TeamID == wstats["WTeamID"]) & (season_stats.Season == r["Season"]), ["Games", "Wins"]] += 1   
    season_stats.loc[(season_stats.TeamID == lstats["LTeamID"]) & (season_stats.Season == r["Season"]), "Games"] += 1   
    




tourney_cresults['ST'] = 'T'
tourney_dresults['ST'] = 'T'
games = pd.concat((season_cresults, tourney_cresults), axis=0, ignore_index=True)
#games = pd.concat((season_dresults, tourney_dresults), axis=0, ignore_index=True)
games.reset_index(drop=True, inplace=True)
games['WLoc'] = games['WLoc'].map({'A': 1, 'H': 2, 'N': 3})

games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)


games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)

games['ScoreDiff'] = games['WScore'] - games['LScore']
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed'] 
games = games.fillna(-1)

#for now
#games = games[games['ST']=='T']

sub['WLoc'] = 3
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['Season'].astype(int)
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] 
sub = sub.fillna(-1)

import matplotlib.pyplot as plt

plt.hist(games["Pred"])
plt.show()

col = [c for c in games.columns if c not in ['ID', 'DayNum', 'ST', 'Team1', 'Team2', 'IDTeams', 'IDTeam1', 'IDTeam2', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'Pred', 'ScoreDiff', 'ScoreDiffNorm', 'WLoc']]

import lightgbm as lgb
import gc
from sklearn.model_selection import StratifiedKFold

gc.enable()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf.get_n_splits(games[col].index, games['Pred'])

test_result = np.zeros(len(sub))

for train_index, test_index in skf.split(games[col].index, games['Pred']):    
    train = games[col]
    y_train = games["Pred"]
    
    train_index += games[col].index[0]
    test_index += games[col].index[0]
    
    X_feat, X_val = train[train.index.isin(train_index)],   train[train.index.isin(test_index)]
    y_feat, y_val = y_train[y_train.index.isin(train_index)], y_train[y_train.index.isin(test_index)]

    model = lgb.LGBMClassifier(max_depth=-1,
                               n_estimators=1000,
                               learning_rate=0.05,
                               colsample_bytree=0.5,
                               objective='binary', 
                               n_jobs=-1)
                               
    model.fit(X_feat, y_feat, eval_metric='binary_error', eval_set=[(X_val, y_val)], 
                  verbose=100, early_stopping_rounds=100)

    test_result += model.predict(sub[col])

    del X_feat, X_val, y_feat, y_val
    gc.collect()

sub['Pred'] = np.round(test_result / 5)

#reg = linear_model.LinearRegression()
#reg.fit(games[col], games['Pred'])
#pred = reg.predict(games[col]).clip(0.000001, 0.999998)
#print('Log Loss:', metrics.log_loss(games['Pred'], pred))
#sub['Pred'] = reg.predict(sub[col]).clip(0.000001, 0.999998)
sub[['ID', 'Pred']].to_csv('submission.csv', index=False)